##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Algoritmos federados personalizados, parte 1: introdução ao Federated Core

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/federated/tutorials/custom_federated_algorithms_1"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/federated/tutorials/custom_federated_algorithms_1.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/federated/tutorials/custom_federated_algorithms_1.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/federated/tutorials/custom_federated_algorithms_1.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

Este tutorial é a primeira parte de uma série de duas partes que demonstra como implementar tipos personalizados de algoritmos federados no TensorFlow Federated (TFF) usando o [Federated Core (FC)](../federated_core.md) - um conjunto de interfaces de nível inferior que servem como base para nossa implementação da camada de [aprendizado federado (FL)](../federated_learning.md).

Essa primeira parte é mais conceitual. Apresentamos alguns dos principais conceitos e abstrações de programação usados no TFF, e demonstramos o uso deles em um exemplo bastante simples com um array distribuído de sensores de temperatura. Na [segunda parte dessa série](custom_federated_algorithms_2.ipynb), usamos os mecanismos apresentados aqui para implementar uma versão simples de algoritmos de treinamento federado e avaliação. Em sequência, recomendamos que estude a [implementação](https://github.com/tensorflow/federated/blob/main/tensorflow_federated/python/learning/algorithms/fed_avg.py) do cálculo federado de médias no `tff.learning`.

No final da série, você deve conseguir reconhecer que os aplicativos do Federated Core não se limitam necessariamente ao aprendizado. As abstrações de programação que oferecemos são bastante genéricas e podem ser usadas, por exemplo, para implementar a análise de dados e outros tipos personalizados de computações sobre dados distribuídos.

Este tutorial foi criado para ser independente, mas recomendamos que você leia primeiro os tutoriais sobre [classificação de imagens](federated_learning_for_image_classification.ipynb) e [geração de texto](federated_learning_for_text_generation.ipynb) para uma introdução mais geral e leve ao framework do TensorFlow Federated e às APIS do [aprendizado federado](../federated_learning.md) (`tff.learning`), ajudando você a contextualizar os conceitos descritos aqui.

## Usos pretendidos

Resumindo, o Federated Core (FC) é um ambiente de desenvolvimento que possibilita expressar de maneira compacta a lógica de programa que combina o código do TensorFlow com operadores de comunicação distribuída, como aqueles usados no [cálculo federado de médias](https://arxiv.org/abs/1602.05629) - computando somas distribuídas, médias e outros tipos de agregações distribuídas em um conjunto de dispositivos do cliente no sistema, transmitindo modelos e parâmetros a esses dispositivos etc.

Talvez você esteja ciente do [`tf.contrib.distribute`](https://www.tensorflow.org/api_docs/python/tf/contrib/distribute) e, neste ponto, é natural se perguntar: qual é a diferença desse framework? Afinal, ambos os frameworks tentam tornar as computações do TensorFlow distribuídas.

Uma forma de pensar sobre isso é: enquanto o objetivo declarado de `tf.contrib.distribute` é *permitir que os usuários usem modelos e código de treinamento existentes com mudanças mínimas para o treinamento distribuído*, e grande parte do foco está em como aproveitar a infraestrutura distribuída para deixar o código de treinamento mais eficiente, o objetivo do Federated Core do TFF é dar aos pesquisadores e profissionais o controle explícito sobre os padrões específicos de comunicação distribuída que eles usarão nos seus sistemas. O foco do FC está em fornecer uma linguagem flexível e extensível para expressar os algoritmos de fluxo de dados distribuídos, em vez de um conjunto concreto de capacidades de treinamento distribuídas.

Um dos principais públicos-alvo para a API FC do TFF são os pesquisadores e profissionais que desejam testar novos algoritmos de aprendizado federado e avaliar as consequências de decisões sutis de design que afetam a maneira como o fluxo de dados é orquestrado no sistema distribuído, sem que fiquem presos em detalhes de implementação do sistema. O nível de abstração que a API FC visa corresponde praticamente ao pseudocódigo que se usaria para descrever a mecânica de um algoritmo de aprendizado federado em uma pesquisa - quais dados existem no sistema e como ele é transformado, mas sem diminuir o nível das trocas de mensagens entre redes ponto a ponto individuais.

O TFF como um todo almeja cenários em que os dados são distribuídos e precisam permanecer assim, por exemplo, por motivos de privacidade e quando coletar todos os dados em uma localização centralizada não é uma opção viável. Isso afeta a implementação de algoritmos de aprendizado de máquina que exigem um maior nível de controle explícito, em comparação com cenários em que todos os dados podem ser acumulados em uma localização centralizada em um data center.

## Antes de começarmos

Antes de mergulhar no código, tente executar o seguinte exemplo "Olá, mundo" para garantir que o ambiente esteja configurado corretamente. Se não funcionar, consulte as instruções no guia de [instalação](../install.md).

In [ ]:
#@test {"skip": true}
!pip install --quiet --upgrade tensorflow-federated

In [ ]:
import collections

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff

In [ ]:
@tff.federated_computation
def hello_world():
  return 'Hello, World!'

hello_world()

b'Hello, World!'

## Dados federados

Uma das características diferenciais do TFF é que ele permite que você expresse de maneira compacta computações baseadas no TensorFlow com *dados federados*. Vamos usar o termo *dados federados* neste tutorial para fazer referência a uma coleção de itens de dados hospedados em um grupo de dispositivos de um sistema distribuído. Por exemplo, aplicativos executados em dispositivos móveis podem coletar dados e armazená-los localmente, sem fazer upload para uma localização centralizada. Ou um array de sensores distribuídos pode coletar e armazenar leituras de temperaturas nas suas localizações.

Os dados federados como esses nos exemplos acima são tratados no TTF como [cidadãos de primeira classe](https://en.wikipedia.org/wiki/First-class_citizen), ou seja, eles aparecem como parâmetros e resultados de funções, além de possuírem tipos. Para reforçar essa noção, vamos chamar os conjuntos de dados federados de *valores federados* ou *valores de tipos federados*.

O ponto importante a entender é que estamos modelando toda a coleção de itens de dados em todos os dispositivos (por exemplo, toda a coleção de leituras de temperaturas de todos os sensores em um array distribuído) como um único valor federado.

Por exemplo, veja como seria definido no TFF o tipo de um *float federado* hospedado por um grupo de dispositivos do cliente. Uma coleção de leituras de temperatura que se materializam em um array de sensores pode ser modelada como um valor desse tipo federado.

In [ ]:
federated_float_on_clients = tff.type_at_clients(tf.float32)

De forma mais geral, um tipo federado no TFF é definido ao especificar o tipo `T` dos *membros constituintes* - os itens de dados que residem nos dispositivos individuais, e o grupo `G` de dispositivos em que os valores federados desse tipo são hospedados (além de uma terceira informação opcional que vamos mencionar a seguir). O grupo `G` de dispositivos que hospeda um valor federado é considerado a *colocação* do valor. Portanto, `tff.CLIENTS` é um exemplo de colocação.

In [ ]:
str(federated_float_on_clients.member)

'float32'

In [ ]:
str(federated_float_on_clients.placement)

'CLIENTS'

Um tipo federado com membros constituintes `T` e uma colocação `G` pode ser representado de maneira compacta como `{T}@G`, conforme mostrado abaixo.

In [ ]:
str(federated_float_on_clients)

'{float32}@CLIENTS'

As chaves `{}` nessa notação concisa servem como um lembrete de que os membros constituintes (itens de dados em diferentes dispositivos) podem variar, conforme você esperaria, por exemplo, em leituras de sensores de temperatura. Então, os clientes como um grupo estão hospedando um [conjunto múltiplo](https://en.wikipedia.org/wiki/Multiset) de itens do tipo `T` que juntos constituem o valor federado.

É importante observar que os membros constituintes de um valor federado são geralmente opacos ao programador, ou seja, um valor federado não deve ser considerado como um simples `dict` com a chave por um identificador de um dispositivo no sistema - esses valores pretendem ser transformados coletivamente somente por *operadores federados* que representam abstratamente vários tipos de protocolos de comunicação distribuída (como agregação). Se parece muito abstrato, não se preocupe - vamos retomar isso em breve e ilustrar com exemplos concretos.

Os tipos federados no TFF tem duas variações: aqueles em que os membros constituintes de um valor federado podem diferir (como acabamos de ver acima) e aqueles em que são todos iguais. Isso é controlado pelo terceiro parâmetro opcional `all_equal` no construtor `tff.FederatedType` (padronizado como `False`).

In [ ]:
federated_float_on_clients.all_equal

False

Um tipo federado com uma colocação `G` em que todos os membros constituintes do tipo `T` são iguais pode ser representado de maneira compacta como `T@G` (em vez de `{T}@G`, ou seja, sem as chaves para refletir o fato de que o conjunto múltiplo de membros constituintes consiste em um único item).

In [ ]:
str(tff.type_at_clients(tf.float32, all_equal=True))

'float32@CLIENTS'

Um exemplo de um valor federado desse tipo que pode surgir em cenários práticos é um hiperparâmetro (como taxa de aprendizado, norma de recorte etc.) que foi transmitido por um servidor a um grupo de dispositivos que participa do treinamento federado.

Outro exemplo é um conjunto de parâmetros para um modelo de aprendizado de máquina pré-treinado no servidor que foram depois transmitidos a um grupo de dispositivos do cliente, onde podem ser personalizados para cada usuário.

Por exemplo, suponha que temos um par de parâmetros `float32` `a` e `b` para um modelo de regressão linear unidimensional. Podemos construir o tipo (não federado) desses modelos para uso no TFF da seguinte maneira. Os colchetes angulares `<>` na string de tipo impressa são uma notação compacta do TFF para tuplas nomeadas ou não.

In [ ]:
simple_regression_model_type = (
    tff.StructType([('a', tf.float32), ('b', tf.float32)]))

str(simple_regression_model_type)

'<a=float32,b=float32>'

Observe que só estamos especificando `dtype`s acima. Os tipos não escalares também são compatíveis. No código acima, `tf.float32` é uma notação de atalho para o `tff.TensorType(dtype=tf.float32, shape=[])` mais geral.

Quando esse modelo é transmitido aos clientes, o tipo do valor federado resultante pode ser representado conforme exibido abaixo.

In [ ]:
str(tff.type_at_clients(
    simple_regression_model_type, all_equal=True))

'<a=float32,b=float32>@CLIENTS'

Em simetria com o *float federado* acima, vamos chamar esse tipo de *tupla federada*. Em termos mais gerais, vamos usar com frequência o termo *XYZ federado* para um valor federado em que os membros constituintes são como *XYZ*. Portanto, vamos falar sobre coisas como *tuplas federadas*, *sequências federadas*, *modelos federados*, e assim por diante.

Agora, voltando a `float32@CLIENTS` - embora apareça replicado em vários dispositivos, é um único `float32`, já que todos os membros são iguais. Em geral, você pode pensar em qualquer tipo federado *todo igual*, ou seja, um de formato `T@G`, como isomórfico a um tipo não federado `T`, já que, em ambos os casos, só há um único (apesar de possivelmente replicado) item de tipo `T`.

Considerando o isomorfismo entre `T` e `T@G`, você pode se perguntar qual é a finalidade, se houver, dos últimos tipos. Continue lendo.

## Colocações

### Visão geral do design

Na seção anterior, apresentamos o conceito de *colocações* - grupos de participantes do sistema que podem hospedar em conjunto um valor federado, e demonstramos o uso de `tff.CLIENTS` como uma especificação de exemplo para uma colocação.

Para explicar o motivo pelo qual a *colocação* é tão fundamental que precisamos incorporá-la ao sistema de tipos do TTF, lembre-se do que mencionamos no início deste tutorial sobre alguns dos usos pretendidos do TFF.

Embora neste tutorial você só verá o código do TFF ser executado localmente em um ambiente simulado, nosso objetivo é que o TFF permite a escrita de código que você pode implantar para a execução em grupos de dispositivos físicos em um sistema distribuído, possivelmente incluindo dispositivos móveis ou embarcados que executam o Android. Cada um desses dispositivos receberia um conjunto separado de instruções para executar localmente, dependendo da função que tem no sistema (um dispositivo de usuário final, um coordenador centralizado, uma camada intermediária em uma arquitetura de vários níveis etc.). É importante conseguir raciocinar sobre quais subconjuntos de dispositivos executam qual código e onde diferentes partes de dados podem se materializar fisicamente.

Isso é especialmente importante ao lidar, por exemplo, com dados de aplicativos em dispositivos móveis. Como os dados são privados e podem ser sensíveis, precisamos da capacidade de verificar estaticamente que esses dados nunca deixarão o dispositivo (e comprovar fatos sobre como os dados estão sendo processados). As especificações de colocação são um dos mecanismos criados para possibilitar isso.

O TFF foi criado como um ambiente de programação focado em dados e, por isso, ao contrário de alguns frameworks existentes que focam em *operações* e onde essas operações podem ser *executadas*, o TFF foca nos *dados*, onde os dados se *materializam* e como eles estão sendo *transformados*. Consequentemente, a colocação é modelada como uma propriedade dos dados no TFF, em vez de uma propriedade das operações nos dados. Você verá na próxima seção que algumas das operações no TFF abrangem várias localizações e são executadas "na rede", por assim dizer, em vez de serem executadas por uma única máquina ou um grupo de máquinas.

A representação do tipo de um determinado valor como `T@G` ou `{T}@G` (em vez de apenas `T`) torna as decisões de colocação de dados explícitas e, junto com a análise estática de programas escritos no TFF, ela pode servir como uma base para fornecer garantias de privacidade formais para dados sensíveis no dispositivo.

No entanto, uma observação importante neste momento é que, embora incentivemos os usuários do TFF a serem explícitos sobre os *grupos* de dispositivos participantes que hospedam os dados (as colocações), o programador nunca lidará com os dados brutos ou as identidades de participantes *individual* individuais.

No corpo do código do TFF, por padrão, não há como enumerar os dispositivos que constituem o grupo representado por `tff.CLIENTS` ou verificar a existência de um dispositivo específico no grupo. Não há conceito de um dispositivo ou identidade de cliente em qualquer lugar da API Federated Core, o conjunto subjacente de abstrações arquitetônicas ou a infraestrutura de runtime principal que fornecemos para apoiar as simulações. Toda a lógica computacional que você escreve será expressa como operações em todo o grupo de clientes.

Lembre-se aqui do que mencionamos antes sobre os valores dos tipos federados não serem como o `dict` do Python, porque não é possível simplesmente enumerar os membros constituintes. Pense em valores que a lógica do seu programa do TFF manipula como sendo associados às colocações (grupos), e não a participantes individuais.

As colocações *são* criadas para serem um cidadão de primeira classe no TFF também e podem aparecer como parâmetros e resultados de um tipo de `colocação` (que será representado por `tff.PlacementType` na API). No futuro, planejamos oferecer uma variedade de operadores para transformar ou combinar colocações, mas está fora do escopo deste tutorial. Por enquanto, basta pensar na `colocação` como um tipo integrado primitivo e opaco no TFF, semelhante a como `int` e `bool` são tipos integrados e opacos no Python, sendo `tff.CLIENTS` uma literal constante desse tipo, como `1` é uma literal constante do tipo `int`.

### Especificação das colocações

O TFF fornece duas literais de colocação básicas, `tff.CLIENTS` e `tff.SERVER`, para facilitar a expressão da rica variedade de cenários práticos que são modelados naturalmente como arquiteturas do servidor do cliente, com vários dispositivos do *cliente* (dispositivos móveis, dispositivos embarcados, bancos de dados distribuídos, sensores etc.) orquestrados por um único coordenador de *servidor* centralizado. O TFF também foi projetado para oferecer suporte a colocações personalizadas, vários grupos de clientes, arquiteturas distribuídas de várias níveis e outras mais gerais, mas discuti-los está fora do escopo deste tutorial.

O TFF não estabelece o que `tff.CLIENTS` ou `tff.SERVER` realmente representa.

Em especial, `tff.SERVER` pode ser um único dispositivo físico (um membro de um grupo único), mas pode ser também um grupo de réplicas em uma replicação máquina de estados que executa clusters tolerante a falhas - não fazemos presunções arquitetônicas especiais. Em vez disso, usamos o bit `all_equal` mencionado na seção anterior para expressar o fato de que geralmente lidamos com apenas um único item de dados no servidor.

Da mesma forma, `tff.CLIENTS` em alguns aplicativos pode representar todos os clientes no sistema - o que no contexto do aprendizado federado às vezes chamados de *população*, mas, por exemplo, nas [implementações de produção do cálculo federado de médias](https://arxiv.org/abs/1602.05629), pode representar uma *coorte* - um subconjunto de clientes selecionados para a participação em uma rodada específica de treinamento. As colocações definidas de maneira abstrata recebem um significado concreto quando uma computação em que elas aparecem é implantada para execução (ou simplesmente invocada como uma função Python em um ambiente simulado, conforme demonstrado neste tutorial). Em nossas simulações locais, o grupo de clientes é determinado pelos dados federados fornecidos como entrada.

## Computações federadas

### Declaração de computações federadas

O TFF foi criado como um ambiente de programação funcional fortemente tipado que é compatível com o desenvolvimento modular.

A unidade básica de composição no TFF é uma *computação federada* - uma seção de lógica que aceita valores federados como entrada e retorna valores federados como saída. Veja como você pode definir uma computação que calcula a média das temperaturas relatadas pelo array de sensores do exemplo anterior.

In [ ]:
@tff.federated_computation(tff.type_at_clients(tf.float32))
def get_average_temperature(sensor_readings):
  return tff.federated_mean(sensor_readings)

Observando o código acima, você talvez esteja se perguntando: já não há construtos de decorador para definir unidades que podem ser compostas, como [`tf.function`](https://www.tensorflow.org/api_docs/python/tf/function) no TensorFlow? E, se sim, por que introduzir mais um e qual é a diferença?

A resposta curta é que o código gerado pelo wrapper `tff.federated_computation` não é *nem* TensorFlow, *nem* Python - é uma especificação de um sistema distribuído em um linguagem *glue* independente de plataforma interna. Neste momento, isso definitivamente soará incompreensível, mas considere essa interpretação intuitiva de uma computação federada como uma especificação abstrata de um sistema distribuído. Vamos explicar isso em um instante.

Primeiro, vamos brincar um pouco com a definição. As computações do TFF são geralmente modeladas como funções - com ou sem parâmetros, mas com assinaturas de tipo bem definidas. Você pode imprimir a assinatura de tipo de uma computação ao consultar sua propriedade `type_signature`, conforme exibido abaixo.

In [ ]:
str(get_average_temperature.type_signature)

'({float32}@CLIENTS -> float32@SERVER)'

A assinatura de tipo nos diz que a computação aceita uma coleção de diferentes leituras de sensores em dispositivos de clientes e retorna uma única média no servidor.

Antes de continuar, vamos refletir sobre isso por um minuto - a entrada e a saída dessa computação estão *em locais diferentes* (em `CLIENTS` x no `SERVER`). Lembre-se do que falamos na seção anterior de colocações sobre como as *operações TFF podem abranger vários locais e serem executadas na rede*, e o que acabamos de falar sobre as computações federadas que representam especificações abstratas de sistemas distribuídos. Acabamos de definir uma computação assim - um sistema distribuído simples em que os dados são consumidos nos dispositivos dos cliente e os resultados agregados surgem no servidor.

Em vários cenários práticos, as computações que representam tarefas de nível superior tendem a aceitar as entradas e relatar as saídas no servidor - isso reflete a ideia de que as computações podem ser acionadas por *consultas* que são originadas e encerradas no servidor.

No entanto, a API FC não impõe esse pressuposto, e vários dos blocos básicos que usamos internamente (incluindo vários operadores `tff.federated_...` que você talvez encontre na API) têm entradas e saídas com colocações distintas. Então, em geral, você não deve pensar em uma computação federada como algo que é *executado no servidor* ou *executado por um servidor*. O servidor é apenas um tipo de participante em uma computação federada. Ao pensar sobre a mecânica dessas computações, é sempre melhor usar a perspectiva de toda a rede global, e não a perspectiva de um único coordenador centralizado.

Em geral, as assinaturas de tipo funcional são representadas de maneira compacta como `(T -> U)` para tipos `T` e `U` de entradas e saídas, respectivamente. O tipo do parâmetro formal (como `sensor_readings` nesse caso) é especificado como o argumento do decorador. Você não precisa especificar o tipo do resultado - ele é determinado automaticamente.

Embora o TFF realmente ofereça formas limitadas de polimorfismo, é recomendável que os programadores sejam explícitos sobre os tipos de dados com que trabalham, já que isso facilita a compreensão, depuração e a verificação formal das propriedades do seu código. Em alguns casos, especificar os tipos explicitamente é um requisito (por exemplo, as computações polimórficas não são executáveis diretamente no momento).

### Execução de computações federadas

Para oferecer suporte ao desenvolvimento e à depuração, o TFF permite que você invoque diretamente as computações definidas dessa forma como funções Python, conforme mostrado abaixo. Quando a computação espera um valor de um tipo federado com o conjunto de bitset `all_equal` como `False`, você pode alimentá-lo como uma simples `list` no Python e, para tipos federados com o bitset `all_equal` como `True`, você pode alimentar diretamente o (único) membro constituinte. Isso também é como os resultados são relatados de novo para você.

In [ ]:
get_average_temperature([68.5, 70.3, 69.8])

69.53334

Ao realizar computações como essa no modo simulação, você age como um observador externo com uma visão de todo o sistema, que tem a capacidade de fornecer entradas e consumir saídas em qualquer localização na rede, como realmente é o caso aqui - você forneceu valores de clientes como entrada e consumiu o resultado do servidor.

Agora, vamos retomar uma observação que fizemos antes sobre o decorador `tff.federated_computation` emitir código em uma linguagem *glue*. A lógica das computações do TFF pode ser expressa como funções comuns em Python (você só precisa decorá-las com `tff.federated_computation`, conforme acabamos de fazer acima), e você pode invocá-las diretamente com argumentos Python, como qualquer outra função Python neste notebook, porém, nos bastidores, as computações do TFF *não* são Python.

Queremos dizer com isso que, quando o interpretador Python se depara com uma função decorada com `tff.federated_computation`, ele faz o tracing das declarações no corpo dessa função uma vez (no tempo de definição) e constrói uma [representação serializada](https://github.com/tensorflow/federated/blob/main/tensorflow_federated/proto/v0/computation.proto) da lógica computacional para uso futuro - seja para execução ou para incorporação como subcomponente em outra computação.

Você pode verificar isso ao adicionar uma declaração print da seguinte maneira:

In [ ]:
@tff.federated_computation(tff.type_at_clients(tf.float32))
def get_average_temperature(sensor_readings):

  print ('Getting traced, the argument is "{}".'.format(
      type(sensor_readings).__name__))

  return tff.federated_mean(sensor_readings)

Getting traced, the argument is "Value".


Você pode pensar no código Python que define uma computação federada semelhante à maneira como você pensaria no código Python que cria um grafo do TensorFlow em um contexto não eager (se você não estiver familiarizado com os usos não eager do TensorFlow, pense no seu código Python definindo um grafo de operações que serão executadas depois, mas sem executá-las dinamicamente). O código de criação de grafo não eager no TensorFlow é Python, mas o grafo do TensorFlow construído por esse código é independente de plataforma e serializável.

De forma semelhante, as computações do TFF são definidas em Python, mas as declarações Python nos seus corpos, como `tff.federated_mean` no exemplo que acabamos de mostrar, são compiladas em uma representação portátil e serializável independente de plataforma em segundo plano.

Como um desenvolvedor, você não precisa se preocupar com os detalhes dessa representação, já que você nunca precisará trabalhar diretamente com ela, mas você deve estar ciente da sua existência, do fato de que as computações do TFF são fundamentalmente não eager e não podem capturar o estado Python arbitrário. O código Python contido no corpo de uma computação do TFF é executado no tempo de definição, quando o corpo da função Python decorada com `tff.federated_computation` passa pelo tracing antes da serialização. O tracing não é realizado novamente no tempo de invocação (exceto quando a função é polimórfica; consulte mais detalhes nas páginas de documentação).

Você pode se perguntar sobre por que escolhemos apresentar uma representação não Python interna dedicada. Um motivo é que, no final, as computação do TFF devem ser implantáveis em ambientes físicos reais e hospedadas em dispositivos móveis ou embarcados, onde Python talvez não esteja disponível.

Outro motivo é que as computações do TFF expressam o comportamento global dos sistemas distribuídos, em vez dos programas Python que expressam o comportamento local de participantes individuais. Você pode ver isso no exemplo simples acima, com o operador especial `tff.federated_mean` que aceita dados em dispositivos de clientes, mas deposita os resultados no servidor.

O operador `tff.federated_mean` não pode ser facilmente modelado como um operador ordinário no Python, já que não é executado localmente - conforme observado antes, ele representa um sistema distribuído que coordena o comportamento de vários participantes do sistema. Vamos chamar esses operadores de *operadores federados* para diferenciá-los dos operadores (locais) normais no Python.

Portanto, o sistema de tipos do TFF e o conjunto fundamental de operações compatíveis com a linguagem do TFF divergem consideravelmente daquelas no Python, exigindo o uso de uma representação dedicada.

### Composição de computações federadas

Conforme observado acima, as computações federadas e os constituintes delas são melhor compreendidos como modelos de sistemas distribuídos, e você pode pensar em compor computações federadas como compor sistemas distribuídos mais complexos a partir de outros mais simples. Você pode considerar o operador `tff.federated_mean` como um tipo de computação federada de modelo integrada com uma assinatura de tipo `({T}@CLIENTS -> T@SERVER)` (realmente, exatamente como as computações que você escreve, esse operador também tem uma estrutura complexa - em segundo plano, dividimos em operadores mais simples).

O mesmo se aplica à composição de computações federadas. A computação `get_average_temperature` pode ser invocada no corpo de outra função Python decorada com `tff.federated_computation` - isso fará com que seja incorporado no corpo do pai, assim como `tff.federated_mean` foi incorporado no próprio corpo antes.

Uma restrição importante de saber é que os corpos das funções Python decoradas com `tff.federated_computation` precisam consistir *somente* em operadores federados, ou seja, eles não podem conter diretamente operações do TensorFlow. Por exemplo, você não pode usar diretamente interfaces `tf.nest` para adicionar um par de valores federados. O código TensorFlow precisa estar confinado em blocos de código decorado com uma `tff.tf_computation` discutida na seção a seguir. Somente envolvido dessa maneira o código do TensorFlow pode ser invocado no corpo de uma `tff.federated_computation`.

Os motivos dessa separação são técnicos (é difícil enganar operadores como `tf.add` para que funcionem com algo além de tensores) e arquitetônicos. A linguagem das computações federadas (ou seja, a lógica construída a partir dos corpos serializados de funções Python decoradas com `tff.federated_computation`) é criada para servir como uma linguagem *glue* independente de plataforma. Essa linguagem glue é usada atualmente para criar sistemas distribuídos de seções incorporadas do código TensorFlow (confinadas a blocos de `tff.tf_computation`). Algum dia, antecipamos a necessidade de incorporar seções de outras lógicas diferentes do TensorFlow, como consultas de bancos de dados relacionais que podem representar pipelines de entrada, todos conectados juntos usando a mesma linguagem glue (os blocos de `tff.federated_computation`).

## Lógica do TensorFlow

### Declaração de computações do TensorFlow

O TFF foi criado para uso com o TensorFlow. Portanto, grande parte do código que você escreverá no TFF provavelmente será código TensorFlow ordinário (ou seja, executado localmente). Para usar esse código com o TFF, como observado acima, ele só precisa ser decorado com `tff.tf_computation`.

Por exemplo, veja como podemos implementar uma função que recebe um número e adiciona `0.5` a ele.

In [ ]:
@tff.tf_computation(tf.float32)
def add_half(x):
  return tf.add(x, 0.5)

Novamente, olhando para isso, você talvez se pergunte por que precisamos definir outro decorador `tff.tf_computation` em vez de apenas usar um mecanismo existente como `tf.function`. Ao contrário da seção anterior, aqui estamos lidando com um bloco ordinário de código TensorFlow.

Há alguns motivos para isso, que vão além do escopo deste tutorial, mas vale a pena mencionar o principal:

- Para incorporar blocos básicos reutilizáveis implementados usando código TensorFlow nos corpos de computações federadas, eles precisam atender a determinadas propriedades, como receber tracing e serialização no tempo de definição, ter assinaturas de tipo etc. Isso geralmente exige algum decorador.

Em geral, recomendamos usar os mecanismos nativos do TensorFlow para composição, como `tf.function`, quando possível, já que a maneira exata em que o decorador do TFF interage com as funções eager deve evoluir.

Agora, voltando ao fragmento de código de exemplo acima, a computação `add_half` que acabamos de definir pode ser tratada pelo TFF como qualquer outra computação do TFF. Em especial, ela tem uma assinatura de tipo do TFF.

In [ ]:
str(add_half.type_signature)

'(float32 -> float32)'

Observe que essa assinatura de tipo não tem colocações. As computações do TensorFlow não podem consumir nem retornar tipos federados.

Você agora também pode usar `add_half` como um bloco básico em outras computações. Por exemplo, veja como você pode usar o operador `tff.federated_map` para aplicar o `add_half` pontual a todos os membros constituintes de um float federado nos dispositivos dos clientes.

In [ ]:
@tff.federated_computation(tff.type_at_clients(tf.float32))
def add_half_on_clients(x):
  return tff.federated_map(add_half, x)

In [ ]:
str(add_half_on_clients.type_signature)

'({float32}@CLIENTS -> {float32}@CLIENTS)'

### Execução de computações do TensorFlow

A execução de computações definidas com `tff.tf_computation` segue as mesmas regras que as descritas para `tff.federated_computation`. Elas podem ser invocadas como invocáveis ordinárias no Python da seguinte maneira.

In [ ]:
add_half_on_clients([1.0, 3.0, 2.0])

[<tf.Tensor: shape=(), dtype=float32, numpy=1.5>,
 <tf.Tensor: shape=(), dtype=float32, numpy=3.5>,
 <tf.Tensor: shape=(), dtype=float32, numpy=2.5>]

Novamente, vale a pena destacar que invocar a computação `add_half_on_clients` dessa maneira simula um processo distribuído. Os dados são consumidos nos clientes e retornados nos clientes. Essa computação faz cada cliente realizar uma ação local. Não há `tff.SERVER` mencionado explicitamente nesse sistema (mesmo se, na prática, orquestrar esse processamento talvez envolva um). Pense em uma computação definida dessa forma como conceitualmente análoga à fase `Map` em `MapReduce`.

Além disso, o que falamos na seção anterior sobre as computações do TFF serem serializadas no tempo de definição também é válido para o código `tff.tf_computation` — o corpo em Python de `add_half_on_clients` passa pelo tracing uma vez no tempo de definição. Nas invocações seguintes, o TFF usa a representação serializada.

A única diferença entre os métodos Python decorados com `tff.federated_computation` e aqueles decorados com `tff.tf_computation` é que estes são serializados como grafos do TensorFlow (enquanto os primeiros não podem conter código TensorFlow diretamente incorporado).

Em segundo plano, cada método decorado com `tff.tf_computation` desativa temporariamente a eager execution para permitir que a estrutura computacional seja capturada. Enquanto a eager execution está desativada localmente, você pode usar construtos eager do TensorFlow, AutoGraph, TensorFlow 2.0 etc., desde que escreva a lógica computacional de modo que possa ser serializada corretamente.

Por exemplo, o código a seguir falhará:

In [ ]:
try:

  # Eager mode
  constant_10 = tf.constant(10.)

  @tff.tf_computation(tf.float32)
  def add_ten(x):
    return x + constant_10

except Exception as err:
  print (err)

Attempting to capture an EagerTensor without building a function.


O código acima falha porque `constant_10` já foi construída fora do grafo que `tff.tf_computation` constrói internamente no corpo de `add_ten` durante o processo de serialização.

Por outro lado, é possível invocar funções Python que modificam o grafo atual quando chamadas dentro de uma `tff.tf_computation`:

In [ ]:
def get_constant_10():
  return tf.constant(10.)

@tff.tf_computation(tf.float32)
def add_ten(x):
  return x + get_constant_10()

add_ten(5.0)

15.0

Observe que os mecanismos de serialização no TensorFlow estão evoluindo, e os pormenores de como o TFF serializa computações também devem evoluir.

### Trabalhando com `tf.data.Dataset`s

Como mencionado antes, um recurso único das `tff.tf_computation`s é que elas permitem trabalhar com `tf.data.Dataset`s definidos de maneira abstrata como parâmetros formais pelo seu código. Os parâmetros que serão representados no TensorFlow como datasets precisam ser declarados usando o construtor `tff.SequenceType`.

Por exemplo, a especificação de tipo `tff.SequenceType(tf.float32)` define uma sequência abstrata de elementos float no TFF. As sequências podem conter tensores ou estruturas aninhadas complexas (veremos exemplos em seguida). A representação concisa de uma sequência de itens do tipo `T` é `T*`.

In [ ]:
float32_sequence = tff.SequenceType(tf.float32)

str(float32_sequence)

'float32*'

Suponha que, em nosso exemplo de sensores de temperatura, cada sensor armazene várias leituras de temperatura. Veja como você pode definir uma computação do TFF no TensorFlow que calcula a média das temperaturas em um único dataset local usando o operador `tf.data.Dataset.reduce`.

In [ ]:
@tff.tf_computation(tff.SequenceType(tf.float32))
def get_local_temperature_average(local_temperatures):
  sum_and_count = (
      local_temperatures.reduce((0.0, 0), lambda x, y: (x[0] + y, x[1] + 1)))
  return sum_and_count[0] / tf.cast(sum_and_count[1], tf.float32)

In [ ]:
str(get_local_temperature_average.type_signature)

'(float32* -> float32)'

No corpo de um método decorado com `tff.tf_computation`, os parâmetros formais de um tipo de sequência do TFF são representados apenas como objetos que se comportam como `tf.data.Dataset`, ou sejam, aceitam as mesmas propriedades e métodos (no momento, eles não são implementados como subclasses desse tipo — isso pode mudar à medida que o suporte a datasets no TensorFlow evoluir)

Você pode verificar isso facilmente da seguinte maneira.

In [ ]:
@tff.tf_computation(tff.SequenceType(tf.int32))
def foo(x):
  return x.reduce(np.int32(0), lambda x, y: x + y)

foo([1, 2, 3])

6

Tenha em mente que, ao contrário de `tf.data.Dataset`s ordinários, esses objetos semelhantes a datasets são marcadores de posição. Eles não contêm nenhum elemento, já que representam parâmetros de tipo de sequência abstratos, que serão ligados a dados concretos quando usados em um contexto concreto. O suporte a datasets de marcador de posição definidos abstratamente ainda é um pouco limitado no momento e, na fase inicial do TFF, você talvez se depare com determinadas restrições. Porém, não precisamos nos preocupar com elas neste tutorial (confira mais detalhes nas páginas da documentação).

Ao executar localmente uma computação que aceita uma sequência em um modo de simulação, como neste tutorial, você pode alimentar a sequência como uma lista Python, da maneira abaixo (além de outras formas, por exemplo, como um `tf.data.Dataset` no modo eager, mas vamos começar com algo simples).

In [ ]:
get_local_temperature_average([68.5, 70.3, 69.8])

69.53333

Como todos os outros tipos do TFF, sequências como as definidas acima podem usar o construtor `tff.StructType` para definir estruturas aninhadas. Por exemplo, veja como declarar uma computação que aceita uma sequência de pares `A`, `B` e retorna a soma dos produtos. Incluímos as declarações de tracing no corpo da computação para que você possa ver como a assinatura de tipo do TFF se traduz nos `output_types` e `output_shapes` do dataset.

In [ ]:
@tff.tf_computation(tff.SequenceType(collections.OrderedDict([('A', tf.int32), ('B', tf.int32)])))
def foo(ds):
  print('element_structure = {}'.format(ds.element_spec))
  return ds.reduce(np.int32(0), lambda total, x: total + x['A'] * x['B'])

element_structure = OrderedDict([('A', TensorSpec(shape=(), dtype=tf.int32, name=None)), ('B', TensorSpec(shape=(), dtype=tf.int32, name=None))])


In [ ]:
str(foo.type_signature)

'(<A=int32,B=int32>* -> int32)'

In [ ]:
foo([{'A': 2, 'B': 3}, {'A': 4, 'B': 5}])

26

O suporte ao uso de `tf.data.Datasets` como parâmetros formais ainda é um pouco limitado e está evoluindo, embora funcione em cenários simples como os usados neste tutorial.

## Juntando tudo

Agora, vamos tentar usar nossa computação do TensorFlow novamente em um contexto federado. Imagine que temos um grupo de sensores, cada um com uma sequência local de leituras de temperaturas. Podemos computar a média de temperatura global calculando a média das médias locais dos sensores da seguinte maneira.

In [ ]:
@tff.federated_computation(
    tff.type_at_clients(tff.SequenceType(tf.float32)))
def get_global_temperature_average(sensor_readings):
  return tff.federated_mean(
      tff.federated_map(get_local_temperature_average, sensor_readings))

Essa não é uma média simples de todas as leituras de temperatura locais de todos os clientes, já que exigiria a ponderação das contribuições de diferentes clientes pelo número de leituras que mantêm localmente. Deixamos a atualização do código acima como um exercício para o leitor. O operador `tff.federated_mean` aceita o peso como um segundo argumento opcional (que deve ser um float federado).

Observe também que a entrada de `get_global_temperature_average` vira agora uma *sequência de floats federados*. As sequências federadas são como geralmente representamos dados no dispositivo no aprendizado federado, com elementos em sequência normalmente representando lotes de dados (você verá exemplos disso em breve).

In [ ]:
str(get_global_temperature_average.type_signature)

'({float32*}@CLIENTS -> float32@SERVER)'

Veja como podemos executar localmente a computação em uma amostra de dados em Python. Agora, fornecemos a entrada como uma `list` de `list`s. A lista externa itera os dispositivos no grupo representado por `tff.CLIENTS`, e as internas iteram os elementos na sequência local de cada dispositivo.

In [ ]:
get_global_temperature_average([[68.0, 70.0], [71.0], [68.0, 72.0, 70.0]])

70.0

Isso conclui a primeira parte deste tutorial... Recomendamos que você continue com a [segunda parte](custom_federated_algorithms_2.ipynb).